In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import seaborn as sn
import configparser
import tweepy

In [15]:
config=configparser.ConfigParser()

In [16]:
#se definen las llaves
config.read('config/config.ini')
api_key=config['twitter']['api_key']
api_key_secret=config['twitter']['api_key_secret']
access_token =config['twitter']['access_token']
access_token_secret =config['twitter']['access_token_secret']

In [17]:
#authentication
auth=tweepy.OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token,access_token_secret)
api=tweepy.API(auth)

In [18]:
try:
    user = api.verify_credentials()
    print(user.name)
except:
    print("Error auth")

Josafat


# Insertar término de búsqueda para el Análisis de Sentimiento

In [ ]:
#Se pregunta por la palabra a preguntar
palabra = input("Buscar: ")

In [ ]:
#Se define la cantidad de tweets a capturar
numero_de_Tweets = int(input(u"Número de tweets a capturar: "))

In [ ]:
#Se define el idioma de los tweets a analizar
lenguaje = input("Idioma [es/en]:")

In [ ]:
for tweet in tweepy.Cursor(api.search_tweets,q=palabra,lang=lenguaje).items(10):
    print(tweet)

# Análisis de Sentimiento con TextBlob

In [ ]:
def ObtenerTweets(palabra,num_tweets,lenguaje):
    #Se define las listas que capturan la popularidad
    polaridad_list = []
    numeros_list = []
    numero = 1
    for tweet in tweepy.Cursor(api.search_tweets, q=palabra,tweet_mode='extended', lang=lenguaje).items(num_tweets):
        try:
            #Se toma el texto, se hace el analisis de sentimiento
            #y se agrega el resultado a las listas
            analisis = TextBlob(tweet.text)
            analisis = analisis.sentiment
            polaridad = analisis.polarity
            polaridad_list.append(polaridad)
            numeros_list.append(numero)
            numero = numero + 1

        except tweepy.TweepError as e:
            print(e.reason)

        except StopIteration:
            break
    return (numeros_list,polaridad_list,numero)


# Visualizar Sentimiento y Categorizar

In [ ]:
def GraficarDatos(numeros_list,polaridad_list,numero):
    axes = plt.gca()
    axes.set_ylim([-1, 2])
    
    plt.scatter(numeros_list, polaridad_list)
    polaridadPromedio = (sum(polaridad_list))/(len(polaridad_list))
    polaridadPromedio = "{0:.2f}%".format(polaridadPromedio * 100)
    time  = datetime.now().strftime("A : %H:%M\n El: %m-%d-%y")
    plt.text(0, 1.25, 
             "Sentimiento promedio:  " + str(polaridadPromedio) + "\n" + time, 
             fontsize=12, 
             bbox = dict(facecolor='none', 
                         edgecolor='black', 
                         boxstyle='square, pad = 1'))
    
    plt.title("Sentimientos sobre " + palabra + " en twitter")
    plt.xlabel("Numero de tweets")
    plt.ylabel("Polaridad Sentimiento")
    plt.show()
    

In [ ]:
numeros_list,polaridad_list,numero = ObtenerTweets(palabra,numero_de_Tweets,lenguaje)